In [ ]:
# EM: roll dices
'''
Original version:
Example in paper 'What is the expectation maximization algorithm?'
flip a coin A or B 10 times
    X[i][0] is the number of head thrown
    coin[i] indicates coin A or B
    
X = np.array([
    (5, 5), 
    (9, 1), 
    (8, 2), 
    (4, 6), 
    (7, 3), 
])
dices = np.array([1, 0, 0, 1, 0], dtype=np.int32)

I wrote a harder version: rolling dices. The result of coin flips follow a binomial distribution, 
but dice rolling results follow a multinomial distribution. 
'''

In [23]:
import numpy as np
from scipy.stats import binom, multinomial


num_test = 50
num_flip = 20
n_dices = 4
n_sides = 6

# set seed
np.random.seed(76)

# generate experimental results
X = []
for i in range(num_test):
    # roll dices for 'num_flip' times
    rolls = np.random.randint(1, n_sides + 1, size=20)
    counts = np.bincount(rolls, minlength=n_sides + 1)[1:]
    X.append(counts)
X = np.array(X)

dices = np.random.randint(0, n_dices, size=num_test)

assert len(X) == len(dices)


def expectation_step(X, dices, theta):
    '''
    Designed for multinomial distribution. 
    X    : Dice rolling results.                               Shape (num_test, n_sides)
    dices: X[i] is rolled by dices[i].                         Shape (num_test)
    theta: estimated probabilities on each side of each dice.  Shape (n_dices, n_sides)  
    '''
    n_dices = len(np.unique(dices))
    n_results = len(X[0])
    expectation = np.zeros([n_dices, n_results])
    for i in range(len(X)):
        x = X[i]
        n = x.sum()  # flip n times
        P = multinomial.pmf(x, n, theta)
        P_dice = P / P.sum()  # shape: (n_dices)
        expectation += np.outer(P_dice, x)  # shape: (n_coins, n_results)
    
    return expectation


def maximization_step(expectation):
    '''
    Designed for multinomial distribution
    expectation: shape (n_dices, n_sides)
    '''
    theta = expectation / expectation.sum(axis=1, keepdims=True)

    return theta


num_flip = len(X)
n_dices = len(np.unique(dices))
n_sides = len(X[0])

# 0. random initialization
# theta = np.array([[0.6, 0.4], [0.5, 0.5]])
theta = np.random.rand(n_dices, n_sides)
theta = theta / theta.sum(axis=1, keepdims=True)
assert len(theta) == n_dices
assert len(theta[0]) == n_sides

max_iter = 100

step = 0
for _ in range(max_iter):
    step += 1
    # 1. E-step: expectation
    expectation = expectation_step(X, dices, theta)
    
    # 2. M-step: maximization step
    theta_= theta.copy()
    theta = maximization_step(expectation)
    
    if np.abs((theta - theta_)).max() < 1e-8:
        print(f"Steps: {step}")
        break

print(theta.round(5))


Steps: 67
[[0.1436  0.26223 0.10751 0.10838 0.14795 0.23033]
 [0.22693 0.14073 0.184   0.13814 0.15182 0.15838]
 [0.16981 0.16068 0.14772 0.2375  0.12083 0.16345]
 [0.13902 0.11371 0.19491 0.12634 0.27677 0.14924]]
